<a href="https://www.kaggle.com/code/boshili/use-bi-lstm?scriptVersionId=93514449" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This note is inspired form [tps_apr_lstms_220417](https://www.kaggle.com/code/zhixx018/tps-apr-lstms-220417) and [TPSApr22 - FE + Pseudo Labels + Bi-LSTM](https://www.kaggle.com/code/hasanbasriakcay/tpsapr22-fe-pseudo-labels-bi-lstm)

In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')

## Fetaure Engineering

In [ ]:
def add_lag_features(df):

    sensors = ['sensor_'+'%02d'%i for i in range(0, 13)]
    for sensor in sensors:
        df[sensor + '_lag1'] = df.groupby('sequence')[sensor].shift(1)
        df.fillna(0, inplace=True)
        df[sensor + '_diff1'] = df[sensor] - df[sensor + '_lag1']

    return df

In [ ]:
train = add_lag_features(train)
test = add_lag_features(test)

In [ ]:
groups = train['sequence']

In [ ]:
Window = 60
y = train_labels['state'].to_numpy()
train.drop(["sequence","step","subject"], axis=1, inplace=True)
test.drop(["sequence","step","subject"], axis=1, inplace=True)

In [ ]:
train

## Normalize Data

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(train)
X_train = sc.transform(train)
X_test = sc.transform(test)

## Reshape Data

set 60 row in one sequence

In [ ]:
y_train = y.copy()
X_train = X_train.reshape(-1, Window, X_train.shape[-1])
X_test = X_test.reshape(-1, Window, X_test.shape[-1])

In [ ]:
print(y_train.shape, X_train.shape, X_test.shape)

## Build Bidirectional Model

In [ ]:
from keras import Model
from keras.layers import *
from keras.callbacks import *
from keras.metrics import AUC

In [ ]:
def get_model():

    x_input = Input(shape=X_train.shape[1:])

    x1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
    x2 = Bidirectional(LSTM(units=256, return_sequences=True))(x1)

    z2 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
    z3 = Bidirectional(LSTM(units=256, return_sequences=True))(z2)
    c = Concatenate(axis=2)([x2, z3])

    x3 = Bidirectional(LSTM(units=128, return_sequences=True))(c)

    x4 = GlobalMaxPooling1D()(x3)
    x5 = Dense(units=128, activation='selu')(x4)
    x_output = Dense(1, activation='sigmoid')(x5)

    model = Model(inputs=x_input, outputs=x_output, name='Bi-Lstm')

    model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=[AUC(name = 'auc')])
    return  model


model = get_model()
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True)

## Fit Model

In [ ]:
def plot_hist(hist, metric='auc', ax=None, fold=0):
    if ax==None:
        plt.plot(hist.history[metric])
        plt.plot(hist.history["val_" + metric])
        plt.title(f"model performance fold {fold}")
        plt.ylabel("area_under_curve")
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="upper left")
        plt.show()
        return
    else:
        ax.plot(hist.history[metric])
        ax.plot(hist.history["val_" + metric])
        ax.set_title(f"model performance fold {fold}")
        ax.set_ylabel("area_under_curve")
        ax.set_xlabel("epoch")
        ax.legend(["train", "validation"], loc="upper left")

## Use K-Fold for corss validation

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import gc


def fit_model(nfold, epochs=60, batch_size=32, verbose=False):
    test_preds = []
    auc = []
    ncols = 5 if nfold > 5 else nfold
    nrows = int(round(nfold / ncols))

    col, row = 0, 0
    fig, axes = plt.subplots(nrows, ncols, figsize=(16, round(nrows*16/ncols)))

    kf = GroupKFold(n_splits=nfold)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X_train, y_train, groups.unique())):
        print(f"Fold: {fold+1}", end=' ')
        X_train_part, X_valid = X_train[train_idx], X_train[test_idx]
        y_train_part, y_valid = y_train[train_idx], y_train[test_idx]


        lr = ReduceLROnPlateau(monitor="val_auc", mode='max', factor=0.7, patience=4, verbose=False)
        es = EarlyStopping(monitor='val_auc',mode='max', patience=10, verbose=False,restore_best_weights=True)
        model = get_model()
        history = model.fit(X_train_part, y_train_part, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=batch_size,
                            callbacks=[es,lr], verbose=verbose)

        y_pred = model.predict(X_valid).squeeze()
        auc_score = roc_auc_score(y_valid, y_pred)
        print(f'auc: {round(auc_score, 5)}')
        test_preds.append(model.predict(X_test).squeeze())
        auc.append(auc_score)

        plot_hist(history, metric='auc', ax=axes[col] if nrows <= 1 else axes[row][col], fold=fold+1)
        del X_train_part, X_valid, y_train_part, y_valid, history, model
        gc.collect()

        col += 1
        if col >= ncols:
            row += 1
            col = 0
    return test_preds, auc

folds = 10
(test_preds, auc) = fit_model(folds, epochs=15, batch_size=256)



In [ ]:
print(f"the mean AUC for the {folds} folds is : {round(np.mean(auc)*100,3)}")

In [ ]:
sub_data = pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv")

In [ ]:
sub_data['state'] = sum(test_preds)/folds
sub_data.state = (sub_data.state > 0.5).astype(int)


In [ ]:
sub_data.to_csv('submission.csv', index=False)